In [6]:
# importing libraries

import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from tqdm import tqdm

pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)
pd.set_option('display.max_colwidth',None)


from sklearn.metrics import precision_score #precision
from sklearn.metrics import recall_score  #recall
from sklearn.metrics import auc  #area under curve
from sklearn.metrics import roc_auc_score  
from sklearn.metrics import log_loss  #log loss
from sklearn.metrics import f1_score  #f1 score
from sklearn.metrics import roc_curve #plot roc curve
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing

import warnings
warnings.filterwarnings('ignore')

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.preprocessing.text import tokenizer_from_json
import json
from gensim.models import Word2Vec
from keras.models import load_model

# Load the model
LSTM_model = load_model('my_model.h9')
model = Word2Vec.load("word2vec.model")

# Function to predict sentiment
def tell_sentiment(rev):
    reviews = []
    reviews.append(rev.lower())
    # Loading the tokenizer from a json file
    with open('tokenizer.json') as f:
        data = json.load(f)
        tokenizer = tokenizer_from_json(data)
    sequences_validate = tokenizer.texts_to_sequences(reviews)
    # Define a function to replace words with vectors
    def replace_with_vectors(data):
        vectorized_data = []
        for review in tqdm(data):
            vectorized_review = []
            for word_index in review:
                if word_index in tokenizer.index_word:
                    word = tokenizer.index_word[word_index]
                    if word in model.wv:
                        vectorized_review.append(model.wv[word])
                    else:
                        vectorized_review.append(np.zeros(model.wv.vector_size))  # Default to zero vector if the word is not in the Word2Vec model
                else:
                    vectorized_review.append(np.zeros(model.wv.vector_size))  # Default to zero vector if the word index is not in the tokenizer
            vectorized_data.append(vectorized_review)
        return vectorized_data

    X_test_vectorized = replace_with_vectors(sequences_validate)

    # Pad sequences after vectorization
    max_length=200

    X_validate_padded = pad_sequences(X_test_vectorized, dtype='float32', padding='post', maxlen=max_length)
    predictions = LSTM_model.predict(X_validate_padded)
    if predictions[0][0] >=0.5:
        return "Positive Review"
    else:
        return "Negative Review"
        
    

# Input text widget for movie review
review_input = widgets.Textarea(
    placeholder='Enter movie review here...',
    description='Review:',
    disabled=False
)

# Button to perform sentiment analysis
button = widgets.Button(description="Analyze Sentiment")

# Output widget to display the sentiment
output = widgets.Output()

# Define the function to be called when button is clicked
def on_button_click(b):
    with output:
        review_text = review_input.value
        sentiment = tell_sentiment(review_text)
        print(f'Sentiment: {sentiment}')

# Link button click to the function
button.on_click(on_button_click)

# Display the widgets
display(review_input, button, output)


Textarea(value='', description='Review:', placeholder='Enter movie review here...')

Button(description='Analyze Sentiment', style=ButtonStyle())

Output()